In [1]:
import sys; sys.path.insert(0, '../../')

In [2]:
from pathlib import Path
from models.snn.factory import build_snn
from models.snn.train import train_model, evaluate_model
from models.snn.tuning import tune_hyperparameters
from models.preprocessing import load_preprocessed

2026-01-01 21:43:58.605689: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2026-01-01 21:43:58.606027: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2026-01-01 21:44:01.447151: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2026-01-01 21:44:02.906649: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2026-01-01 21:44:02.906771: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: l

In [3]:
splits, class_names = load_preprocessed('../../data/processed/')
X_train, y_train = splits['train']
X_val, y_val = splits['val']
X_test, y_test = splits['test']

Loaded train: X=(247, 614400), y=(247, 10)
Loaded val: X=(71, 614400), y=(71, 10)
Loaded test: X=(36, 614400), y=(36, 10)
Loaded class names: ['benjamin', 'christian', 'felix', 'jonas', 'leon', 'mark', 'marvin', 'ohman', 'veronica', 'yannes']


In [4]:
checkpoint_dir = Path('../../checkpoints/snn')
env_path = str((checkpoint_dir / "best_hparams.env").absolute())

In [ ]:
best_config, analysis = tune_hyperparameters(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    n_features=X_train.shape[1],
    n_classes=len(class_names),
    max_epochs=5,
    num_samples=20,
    project_name="snn_ray_tune",
 )

[Ray Tune] Initializing Ray runtime...


2026-01-01 21:44:06,272	INFO worker.py:1927 -- Started a local Ray instance.
2026-01-01 21:44:06,643	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


[Ray Tune] Preparing hyperparameter search...
  max_epochs      = 5
  num_samples     = 20
  project_name    = snn_ray_tune
[Ray Tune] Starting tuning run...


:job_id:01000000
:task_name:bundle_reservation_check_func


:job_id:01000000
:task_name:bundle_reservation_check_func


...

|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (0%)                    | ETA:  --:--:--
|######               Building network (10%)                     | ETA: 0:00:06
|######               Building network (10%)                     | ETA: 0:00:07
|######               Building network (10%)                     | ETA: 0:00:07
|######               Building network (10%)                     | ETA: 0:00:08
|#####################Building network (40%)                     | ETA: 0:00:01
|#####################Building network (40%)                     | ETA: 0:00:01
|#####################Building network (40%)                     | ETA: 0:00:01
|#####################Building network (

2026-01-01 21:46:43.099291: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2026-01-01 21:46:43.099663: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-PH8A95A): /proc/driver/nvidia/version does not exist
2026-01-01 21:46:43.100521: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


|     #                   Constructing graph                          | 0:00:00
|           Constructing graph: pre-build stage (0%)           | ETA:  --:--:--
Constructing graph: pre-build stage finished in 0:00:00
|             Constructing graph: build stage (0%)             | ETA:  --:--:--
|####          Constructing graph: build stage (7%)              | ETA: 0:00:00
|##############Constructing graph: build stage (35%)             | ETA: 0:00:00
|##############Constructing graph: build stage (64%)             | ETA: 0:00:00
|##############Constructing graph: build stage (85%)###          | ETA: 0:00:00
|############Constructing graph: build stage (100%)############| ETA:  00:00:00
|        #                Constructing graph                          | 0:00:00
Constructing graph: build stage finished in 0:00:00
Construction finished in 0:00:00
Compiling model...

Training for 5 epochs...
  Input shape: (247, 1, 614400)
  Label shape: (247, 1, 10)
Epoch 1/5
|           Constructing

/home/raah/.local/share/virtualenvs/neuro-ai-9gzpYvh7/lib/python3.10/site-packages/nengo_dl/simulator.py:1003: UserWarning: Running for one timestep, but the network contains synaptic filters (which will introduce at least a one-timestep delay); did you mean to set synapse=None?
  warnings.warn(


|####          Constructing graph: build stage (7%)              | ETA: 0:00:00
|##############Constructing graph: build stage (35%)             | ETA: 0:00:00
|##############Constructing graph: build stage (50%)             | ETA: 0:00:00
|##############Constructing graph: build stage (71%)             | ETA: 0:00:00
|############Constructing graph: build stage (100%)############| ETA:  00:00:00
Constructing graph: build stage finished in 0:00:00
|           Constructing graph: pre-build stage (0%)           | ETA:  --:--:--
Constructing graph: pre-build stage finished in 0:00:00
|             Constructing graph: build stage (0%)             | ETA:  --:--:--
|####          Constructing graph: build stage (7%)              | ETA: 0:00:00
|##############Constructing graph: build stage (35%)             | ETA: 0:00:00
|##############Constructing graph: build stage (57%)             | ETA: 0:00:00
|##############Constructing graph: build stage (78%)             | ETA: 0:00:00
|###########

In [ ]:
print("\nBest hyperparameters found by Ray Tune:\n")
print(f"  Hidden neurons: {best_config['n_neurons_hidden']}")
print(f"  Synapse fast: {best_config.get('synapse_fast')}")
print(f"  Synapse slow: {best_config.get('synapse_slow')}")
print(f"  Learning rate: {best_config['learning_rate']:.2e}")
print(f"  Batch size: {best_config['batch_size']}")

best_hidden = int(best_config['n_neurons_hidden'])
best_syn_fast = float(best_config.get('synapse_fast'))
best_syn_slow = float(best_config.get('synapse_slow'))
best_lr = float(best_config['learning_rate'])
best_batch_sz = int(best_config['batch_size'])


Best hyperparameters found by Ray Tune:

  Hidden neurons: 167
  Synapse fast: 0.0044048588944318855
  Synapse slow: 0.05422395713610029
  Learning rate: 1.10e-03
  Batch size: 32


In [ ]:
print(f"\nSaving best hyperparameters to {env_path} ...")
with open(env_path, "w") as f:
    f.write(f"N_NEURONS_HIDDEN={best_hidden}\n")
    f.write(f"SYNAPSE_FAST={best_syn_fast}\n")
    f.write(f"SYNAPSE_SLOW={best_syn_slow}\n")
    f.write(f"LEARNING_RATE={best_lr}\n")
    f.write(f"BATCH_SIZE={best_batch_sz}\n")
    f.write(f"N_FEATURES={X_train.shape[1]}\n")
    f.write(f"N_CLASSES={len(class_names)}\n")
print("Best hyperparameters saved.\n")


Saving best hyperparameters to /home/z003dy4u/repos/neuroAi/notebooks/snn/../../checkpoints/snn/best_hparams.env ...
Best hyperparameters saved.

